In [57]:
from itertools import islice
from textwrap import dedent
import griffe
from griffe import Alias, Function, ParameterKind, Parser
import subprocess
inspect_ai = griffe.load("inspect_ai", docstring_parser=Parser.google)

sha = subprocess.run(["git", "rev-parse", "HEAD"],capture_output=True).stdout.decode().strip()

def read_lines(filename, start_line, end_line) -> list[str]:
    with open(filename, 'r') as file:
        return list(islice(file, start_line-1, end_line))

def format_code(lines: list[str]) -> str:
    code = "".join(lines)
    return code.removesuffix(":\n")


def render_function(function: Function) -> str:
    
    

    # validate that we have a docstring
    if function.docstring is None:
        raise ValueError(f"Function {function.name} does not have a docstring.")

    sections = function.docstring.parse()

    # url to code
    url = f"https://github.com/UKGovernmentBEIS/inspect_ai/blob/{sha}/src/{function.relative_package_filepath}#L{function.lineno}"

    # read function source code
    code_fmt = format_code(read_lines(function.filepath, function.lineno, function.docstring.lineno-1))

    # build param list
    params: list[str] = []
    for param in function.parameters:
        param_fmt = f"- {param.name} ({param.annotation.modernize()}"
        if param.required:
            param_fmt = f"{param_fmt})"
        else:
            param_fmt = f"{param_fmt}, default: {param.default})" 
    
        if param.kind == ParameterKind.var_positional:
            param_fmt = f"*{param_fmt}"
        elif param.kind == ParameterKind.var_keyword:
            param_fmt = f"**{param_fmt}"    

        params.append(param_fmt)
    params_fmt = "\n".join(params)

    return f"{function.name}\n\n{url}\n\n{code_fmt}\n\n{params_fmt}\n\n{function.as_json(indent=2)}"

def render_reference(path: str) -> str:
    # lookup object
    object = inspect_ai
    for segment in path.split("."):
        object = object.members[segment]

    # resolve aliases
    if isinstance(object, Alias):
        object = object.final_target
   
    # render
    if isinstance(object, Function):
        return render_function(object)
    else:
        raise ValueError(f"Reference object type ({type(object)}) for {path} is unsupported.")
   
    
       
    

In [58]:
print(render_reference("solver.basic_agent"))

None
None
None
None
None
None
None
None
None
None
None
None
None
basic_agent

https://github.com/UKGovernmentBEIS/inspect_ai/blob/2c7deb4a58981c5470f562865533a4a0c226857a/src/inspect_ai/solver/_basic_agent.py#L50

@solver
def basic_agent(
    *,
    init: Solver | list[Solver] | None = None,
    tools: list[Tool] | Solver | None = None,
    cache: bool | CachePolicy = False,
    max_attempts: int = 1,
    message_limit: int | None = None,
    token_limit: int | None = None,
    max_tool_output: int | None = None,
    score_value: ValueToFloat | None = None,
    incorrect_message: str
    | Callable[
        [TaskState, list[Score]], str | Awaitable[str]
    ] = DEFAULT_INCORRECT_MESSAGE,
    continue_message: str = DEFAULT_CONTINUE_MESSAGE,
    submit_name: str = DEFAULT_SUBMIT_NAME,
    submit_description: str = DEFAULT_SUBMIT_DESCRIPTION,
    **kwargs: Unpack[BasicAgentDeprecatedArgs],
) -> Solver

- init (Solver | list[Solver] | None, default: None)
- tools (list[Tool] | Solver | No